In [140]:
# 準備工作

from bs4 import BeautifulSoup
import time
import datetime
import re
import sqlite3
import os
import json
import pprint

wsUrl = "http://addr.tgos.nat.gov.tw/addrws/v30/QueryAddr.asmx/QueryAddr"

AppID = "pwP6iYU+Y3khg+UtIfgFVrYFcgpILUNw1k9Qpq5uG5woDEUPbj+xwg=="

APIKey = "cGEErDNy5yNr14zbsE/4GSfiGP5i3PuZPka9hc5dBrghK3prMDRxrW+ej1/84C24MCPQIrith6c/YNkx/wcZej/0NNDi0o9Onq/3Kb/B5vfFQbK9iyQvAI4p3AQbuCTe7OFmtsx6oL/IWORJXpg9QaSkugXEsEhTJBvFEetKLTJCUwKP7TS0Wh3oIr4N5tU55hEjhsSiDUVqQobQ282b2UXsUbEVqcfu7EdkqdqMDTANaOfjNfEiyE5m2L9Jf/VW7eNV5LCOzpf/4nxqTFQDnMm7mck2CBAxvI4goDJI2Cv8IuNPGLdWjBAQh6+Y/VEXzoxCOs0Fgzo="

jsUrl_tgosMap = "http://api.tgos.nat.gov.tw/TGOS_API/tgos?ver=2&AppID=" + AppID + "&APIKey=" + APIKey

url = "https://www.tgos.tw/TGOS_WEB_API/Sample_Codes/TGOSQueryAddr/QueryAddrTGOSMap.aspx"
viewstate = "/wEPDwULLTEwNDI1NTA0NjAPZBYCAgMPZBYCAgcPDxYCHgRUZXh0BZEKew0KICAiSW5mbyI6IFsNCiAgICB7DQogICAgICAiSXNTdWNjZXNzIjogIlRydWUiLA0KICAgICAgIkluQWRkcmVzcyI6ICLoh7rljJfluILkuK3lsbHljYDmnb7msZ/ot680Njnlt7c06JmfIiwNCiAgICAgICJJblNSUyI6ICJFUFNHOjM4MjYiLA0KICAgICAgIkluRnV6enlUeXBlIjogIuacgOi/kemWgOeJjOiZn+apn+WItiIsDQogICAgICAiSW5GdXp6eUJ1ZmZlciI6ICIwIiwNCiAgICAgICJJbklzT25seUZ1bGxNYXRjaCI6ICJGYWxzZSIsDQogICAgICAiSW5Jc0xvY2tDb3VudHkiOiAiRmFsc2UiLA0KICAgICAgIkluSXNMb2NrVG93biI6ICJGYWxzZSIsDQogICAgICAiSW5Jc0xvY2tWaWxsYWdlIjogIkZhbHNlIiwNCiAgICAgICJJbklzTG9ja1JvYWRTZWN0aW9uIjogIkZhbHNlIiwNCiAgICAgICJJbklzTG9ja0xhbmUiOiAiRmFsc2UiLA0KICAgICAgIkluSXNMb2NrQWxsZXkiOiAiRmFsc2UiLA0KICAgICAgIkluSXNMb2NrQXJlYSI6ICJGYWxzZSIsDQogICAgICAiSW5Jc1NhbWVOdW1iZXJfU3ViTnVtYmVyIjogIkZhbHNlIiwNCiAgICAgICJJbkNhbklnbm9yZVZpbGxhZ2UiOiAiVHJ1ZSIsDQogICAgICAiSW5DYW5JZ25vcmVOZWlnaGJvcmhvb2QiOiAiVHJ1ZSIsDQogICAgICAiSW5SZXR1cm5NYXhDb3VudCI6ICIwIiwNCiAgICAgICJPdXRUb3RhbCI6ICIxIiwNCiAgICAgICJPdXRNYXRjaFR5cGUiOiAi5a6M5YWo5q+U5bCNIiwNCiAgICAgICJPdXRNYXRjaENvZGUiOiAiW+iHuuWMl+W4gl1cdEZVTEw6MSIsDQogICAgICAiT3V0VHJhY2VJbmZvIjogIlvoh7rljJfluIJdXHQgeyDlrozlhajmr5TlsI0gfSDmib7liLDnrKblkIjnmoTploDniYzlnLDlnYAiDQogICAgfQ0KICBdLA0KICAiQWRkcmVzc0xpc3QiOiBbDQogICAgew0KICAgICAgIkZVTExfQUREUiI6ICLoh7rljJfluILkuK3lsbHljYDooYzmlL/ph4wx6YSw5p2+5rGf6LevNDY55be3NOiZnyIsDQogICAgICAiQ09VTlRZIjogIuiHuuWMl+W4giIsDQogICAgICAiVE9XTiI6ICLkuK3lsbHljYAiLA0KICAgICAgIlZJTExBR0UiOiAi6KGM5pS/6YeMIiwNCiAgICAgICJORUlHSEJPUkhPT0QiOiAiMemEsCIsDQogICAgICAiUk9BRCI6ICLmnb7msZ/ot68iLA0KICAgICAgIlNFQ1RJT04iOiAiIiwNCiAgICAgICJMQU5FIjogIjQ2OeW3tyIsDQogICAgICAiQUxMRVkiOiAiIiwNCiAgICAgICJTVUJfQUxMRVkiOiAiIiwNCiAgICAgICJUT05HIjogIiIsDQogICAgICAiTlVNQkVSIjogIjTomZ8iLA0KICAgICAgIlgiOiAzMDM4OTMuNDE4LA0KICAgICAgIlkiOiAyNzczMjI4Ljg1Mw0KICAgIH0NCiAgXQ0KfWRkZAY3e+XhI03mhPhoUmPVOEj4/WrDVk9X4Mkg7snEawXK"
viewstategen = '55FF419D'
eventval = ":/wEdAAMSiffgTVFhMlMWIRMX7yLEzh9mL6dLWI6NyCvVUGLeEOh2ZUG1jMhoL4US5tgdjir/WxzC6SOzlkkYTjDCLNtYu+Nyb3dUQ60snn+jvXIz6A=="
address = "承德路4段175號"

params = {
#           '__VIEWSTATE':viewstate,
          '__VIEWSTATEGENERATOR':viewstategen,
#           '__EVENTVALIDATION': eventval,
          'TxtAddress':address,
          'BtnQuery':'查詢'
         }


params2 = {
            # 定位參數可依所需自行設定，以下設定最常用的，說明可查 
            # http://addr.tgos.nat.gov.tw/addrws/v30/QueryAddr.asmx?op=QueryAddr ***********************/
            'oAPPId': AppID, # 應用程式識別碼(APPId)
            'oAPIKey': APIKey, # 應用程式介接驗證碼(APIKey)
            'oAddress': address, # 所要查詢的門牌位置
            'oSRS': 'EPSG:3826', # 回傳的坐標系統
            'oFuzzyType': "2", # 模糊比對的代碼
            'oResultDataType': 'json', # 回傳的資料格式
            'oFuzzyBuffer': "0", # 模糊比對回傳門牌號的許可誤差範圍
            'oIsOnlyFullMatch': "false", # 是否只進行完全比對
            'oIsLockCounty': "false", # 是否鎖定縣市
            'oIsLockTown': "false", # 是否鎖定鄉鎮市區
            'oIsLockVillage': "false", # 是否鎖定村里
            'oIsLockRoadSection': "false", # 是否鎖定路段
            'oIsLockLane': "false", # 是否鎖定巷
            'oIsLockAlley': "false", # 是否鎖定弄
            'oIsLockArea': "false", # 是否鎖定地區
            'oIsSameNumber_SubNumber': "false", # 號之、之號是否視為相同
            'oCanIgnoreVillage': "false", # 找不時是否可忽略村里
            'oCanIgnoreNeighborhood': "false", # 找不時是否可忽略鄰
            'oReturnMaxCount': "0"# 如為多筆時，限制回傳最大筆數
}



# header = {'url': wsUrl, # API 服務網址
#           'type': 'GET', # 請求模式使用 GET 模式
#           'data': 
#              {
#                 # 定位參數可依所需自行設定，以下設定最常用的，說明可查 
#                 # http://addr.tgos.nat.gov.tw/addrws/v30/QueryAddr.asmx?op=QueryAddr ***********************/
#                 'oAPPId': AppID, # 應用程式識別碼(APPId)
#                 'oAPIKey': APIKey, # 應用程式介接驗證碼(APIKey)
#                 'oAddress': address, # 所要查詢的門牌位置
#                 'oSRS': 'EPSG:3826', # 回傳的坐標系統
#                 'oFuzzyType': "2", # 模糊比對的代碼
#                 'oResultDataType': 'json', # 回傳的資料格式
#                 'oFuzzyBuffer': "0", # 模糊比對回傳門牌號的許可誤差範圍
#                 'oIsOnlyFullMatch': "false", # 是否只進行完全比對
#                 'oIsLockCounty': "false", # 是否鎖定縣市
#                 'oIsLockTown': "false", # 是否鎖定鄉鎮市區
#                 'oIsLockVillage': "false", # 是否鎖定村里
#                 'oIsLockRoadSection': "false", # 是否鎖定路段
#                 'oIsLockLane': "false", # 是否鎖定巷
#                 'oIsLockAlley': "false", # 是否鎖定弄
#                 'oIsLockArea': "false", # 是否鎖定地區
#                 'oIsSameNumber_SubNumber': "false", # 號之、之號是否視為相同
#                 'oCanIgnoreVillage': "false", # 找不時是否可忽略村里
#                 'oCanIgnoreNeighborhood': "false", # 找不時是否可忽略鄰
#                 'oReturnMaxCount': "0"# 如為多筆時，限制回傳最大筆數
#              }
#          }

In [141]:
wsUrl_all = [str(each) + "=" + str(params2[each]) for each in params2]
wsUrl_all = wsUrl + "?ver=2&" + "&".join(wsUrl_all)
wsUrl_all

'http://addr.tgos.nat.gov.tw/addrws/v30/QueryAddr.asmx/QueryAddr?ver=2&oIsLockCounty=false&oFuzzyBuffer=0&oAPPId=pwP6iYU+Y3khg+UtIfgFVrYFcgpILUNw1k9Qpq5uG5woDEUPbj+xwg==&oReturnMaxCount=0&oIsLockVillage=false&oIsSameNumber_SubNumber=false&oAPIKey=cGEErDNy5yNr14zbsE/4GSfiGP5i3PuZPka9hc5dBrghK3prMDRxrW+ej1/84C24MCPQIrith6c/YNkx/wcZej/0NNDi0o9Onq/3Kb/B5vfFQbK9iyQvAI4p3AQbuCTe7OFmtsx6oL/IWORJXpg9QaSkugXEsEhTJBvFEetKLTJCUwKP7TS0Wh3oIr4N5tU55hEjhsSiDUVqQobQ282b2UXsUbEVqcfu7EdkqdqMDTANaOfjNfEiyE5m2L9Jf/VW7eNV5LCOzpf/4nxqTFQDnMm7mck2CBAxvI4goDJI2Cv8IuNPGLdWjBAQh6+Y/VEXzoxCOs0Fgzo=&oAddress=承德路4段175號&oCanIgnoreVillage=false&oSRS=EPSG:3826&oIsLockRoadSection=false&oIsLockTown=false&oIsLockLane=false&oCanIgnoreNeighborhood=false&oFuzzyType=2&oIsLockAlley=false&oResultDataType=json&oIsLockArea=false&oIsOnlyFullMatch=false'

In [142]:
test_req = requests.get(wsUrl_all)
test_req.text

'<?xml version="1.0" encoding="utf-8"?>\r\n<string xmlns="http://tempuri.org/">Base-64 字元陣列或字串的無效長度。</string>'

In [81]:
session = requests.Session()
result = session.post(url,data=params)

In [72]:
result = BeautifulSoup(result.text,'html.parser')
pprint.pprint(result)
# params


<!DOCTYPE html >

<html xmlns="http://www.w3.org/1999/xhtml">
<head id="Head1"><title>
	ASP.NET呼叫應用範例
</title>
<style type="text/css">
        .rowStyle
        {
            padding: 6px 20px;
            line-height: 24px;
            white-space: nowrap;
            letter-spacing: 3px;
            cursor: pointer;
            color: #0097C6;
            font-size: 11pt;
            text-align: left;
        }
        .rowStyle:link
        {
            text-decoration: none;
        }
        .rowStyle:hover
        {
            background-color: rgb(112, 207, 236);
            color: White;
        }
    </style>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<script src="//code.jquery.com/jquery-latest.js" type="text/javascript"></script>
<script charset="utf-8" src="//api.tgos.tw/TGOS_API/tgos?ver=2&amp;AppID=fCY1EqcXsHCeUMV5MsVy/UTcpZ0YwSLV&amp;APIKey=rIdTgl9dz7b7RfcnlObHA71UQON1hpmDvncTV6ayIbo0InpqWVHs7Lqz+7gNm48zYT/WgcG1Klo1OKkB5Loks7xo8ez2NNCCqkIkrQu

In [73]:
# session = requests.Session()
# result = session.get(jsUrl_tgosMap,headers=(headers))

In [ ]:
# QueryAddrTGOSMapSample.js 的程式


		/*
		1.宣告 API 服務網址、APPID與APIKEY
        2.動態載入 TGOS MAP API
		*/
		
        //全國門牌位置比對服務 WebService 網址   
		var wsUrl = "http://addr.tgos.nat.gov.tw/addrws/v30/QueryAddr.asmx/QueryAddr";  
		
		//應用程式識別碼(APPId)
		//ha0HBfvgAgU=
        var AppID = "";  

		//應用程式介接驗證碼(APIKey)
		//tB2ldEWISWO0+NkYyo/ecfuum1bJEIfwWrVmfavznvUU7gn8JXhfkj61mLdS6UUlr7pB0sHBt52e+QbbiZe57Gj0mWZS0bUf9fUVt7YLkd7v4UOwT8nAp+v5eT14eoTDsVjrUbIqSC5VQnJFh8/4tA==
        var APIKey = "";  

		//TGOS MAP API 的 js 檔網址，並須加入 APPId 與 APIKey 參數        
        var jsUrl_tgosMap = "http://api.tgos.nat.gov.tw/TGOS_API/tgos?ver=2&AppID=" + AppID + "&APIKey=" + APIKey; 
		
		//執行 TGOS MAP API 的 js檔載入
        document.write(unescape("%3Cscript type='text/javascript' src='" + jsUrl_tgosMap + "' %3E%3C/script%3E")); 
		
		$(function () {//網頁元素準備好時呼叫(可以選取網頁中的元素，並對操作它們)


        });

        /*==========================================
        執行地址比對並輸出結果至畫面 
        ============================================*/
        function executeLocate(address) {
            var $ResultList = $('#ResultList');
            var $ResultInfo = $('#ResultInfo');
            $.ajax({
                url: wsUrl, ////API 服務網址
                type: 'GET', //請求模式使用 GET 模式
                data: {/****************** 定位參數可依所需自行設定，以下設定最常用的，說明可查 http://addr.tgos.nat.gov.tw/addrws/v30/QueryAddr.asmx?op=QueryAddr ***********************/
                    oAPPId: AppID, //應用程式識別碼(APPId)
                    oAPIKey: APIKey, // 應用程式介接驗證碼(APIKey)
                    oAddress: address, //所要查詢的門牌位置
                    oSRS: 'EPSG:3826', //回傳的坐標系統
                    oFuzzyType: "2", //模糊比對的代碼
                    oResultDataType: 'json', //回傳的資料格式
                    oFuzzyBuffer: "0", //模糊比對回傳門牌號的許可誤差範圍
                    oIsOnlyFullMatch: "false", //是否只進行完全比對
                    oIsLockCounty: "false", //是否鎖定縣市
                    oIsLockTown: "false", //是否鎖定鄉鎮市區
                    oIsLockVillage: "false", //是否鎖定村里
                    oIsLockRoadSection: "false", //是否鎖定路段
                    oIsLockLane: "false", //是否鎖定巷
                    oIsLockAlley: "false", //是否鎖定弄
                    oIsLockArea: "false", //是否鎖定地區
                    oIsSameNumber_SubNumber: "false", //號之、之號是否視為相同
                    oCanIgnoreVillage: "false", //找不時是否可忽略村里
                    oCanIgnoreNeighborhood: "false", //找不時是否可忽略鄰
                    oReturnMaxCount: "0"//如為多筆時，限制回傳最大筆數
                },
                beforeSend: function () {
                    /*===== 執行前會進來 =====*/
                    
                }
            }).done(function (data) {
                /*===== 成功時會進來 =====*/

                //console.log($(data).text());
                var jsonObj = $.parseJSON($(data).text()); //將查詢回來的結果轉成 JSON 物件
                console.log(jsonObj);

                

            }).fail(function (jqXHR, textStatus, errorThrown) {
                /*===== 失敗時會進入 =====*/
                
            }).always(function (data, textStatus, errorThrown) {
                /*===== 不管失敗或成功，最後都會進入 =====*/
            });

        }